# Symbolic Synthesis of a Robot Controller (Genetic Programming)

This notebook explores **Genetic Programming (GP)** to evolve an interpretable controller for a grid-world robot, using **multi-objective optimization** (e.g., distance-to-goal vs. controller complexity).

Repository overview: see `README.md` and `docs/NOTEBOOK.md`.


### Objetivo general

Evolucionar programas simbólicos (árboles) que controlen a un "robot" en un entorno 2D simulado (como seguir una línea o evitar obstáculos), maximizando precisión y minimizando complejidad del controlador.

##### **🧠 ¿Qué es Programación Genética (GP)?**




La GP es una técnica de inteligencia artificial inspirada en la evolución biológica, en la que se evolucionan programas (en forma de árboles de expresiones) en lugar de vectores numéricos como en otros algoritmos genéticos. Los árboles contienen operaciones como +, -, *, condicionales, comparaciones, etc.

##### Definir el árbol de Programación Genética (GP)

**¿Qué es un árbol GP?**

En Programación Genética (GP), cada individuo es un programa en forma de árbol. Este árbol está compuesto por:

- Nodos internos: funciones que operan sobre datos (como suma, resta, condiciones, etc.).

- Hojas o terminales: entradas (como x, y) o constantes (0, 1).

Los individuos serán programas simbólicos en forma de árboles. Cada árbol recibe como entrada las coordenadas (x, y) del robot y devuelve una acción (por ahora: 0 = avanzar, 1 = girar).

##### Formato del controlador (GP):

Usaremos árboles con:

- Funciones internas:

  - if_then_else, add, sub, gt (mayor que), mul

- Terminales:

  - Variables como x, y (posición)

  - Constantes aleatorias


##### Especificaciones del entorno:

- Representaremos el entorno como una rejilla 2D (por ejemplo, 10x10).

- El robot empieza en una posición y debe llegar a una meta.

- El "controlador" decidirá qué acción tomar basándose en la posición o sensores.

- Acciones posibles: avanzar, girar, retroceder.

### Implementación básica

**Instalar y preparar librerías**

In [ ]:
!pip install deap --quiet

In [ ]:
import operator
import math
import random
from deap import base, creator, gp, tools, algorithms
import matplotlib.pyplot as plt
import numpy as np

In [ ]:
# Funciones básicas para componer el árbol
def if_then_else(condition, out1, out2):
    # Esta función implementa una condicional:
    # Si 'condition' es verdadera, devuelve 'out1'; si no, devuelve 'out2'.
    return out1 if condition else out2

📌 Esta función se usará como un nodo en el árbol GP, permitiendo a los programas tener estructuras condicionales del tipo "si-entonces-sino".

In [ ]:
# Registramos el "tipo" de expresión simbólica: una función que devuelve un entero (la acción)
pset = gp.PrimitiveSet("MAIN", 2)  # Dos entradas: x, y

📌 Se define un conjunto de primitivas llamado "MAIN" que acepta 2 argumentos (posicionales). Son las entradas del programa a evolucionar (por ejemplo, pueden ser coordenadas o características del entorno).

In [ ]:
# Añadir funciones primitivas al conjunto
pset.addPrimitive(operator.add, 2)      # Suma: x + y
pset.addPrimitive(operator.sub, 2)      # Resta: x - y
pset.addPrimitive(operator.mul, 2)      # Multiplicación: x * y
pset.addPrimitive(operator.gt, 2)       # Mayor que: x > y (devuelve booleano)
pset.addPrimitive(if_then_else, 3)      # Estructura condicional ternaria

📌 Aquí se agregan nodos funcionales (también llamados primitivas) que el GP puede usar:

add, sub, mul: funciones aritméticas básicas.

gt: compara dos valores y devuelve un booleano.

if_then_else: permite tomar decisiones condicionales dentro del árbol.

👉 Esto permite construir árboles como:


```
if x > y:
    return x + y
else:
    return x * y
```



In [ ]:
# Terminales constantes que pueden usarse en las expresiones
pset.addTerminal(1)
pset.addTerminal(0)

📌 Los terminales son los valores finales del árbol. Aquí se agregan constantes literales que los programas pueden usar.

In [ ]:
# Renombrar las variables de entrada para que sean más legibles
pset.renameArguments(ARG0="x")
pset.renameArguments(ARG1="y")

📌 Por defecto, los nombres de los argumentos son ARG0, ARG1, etc. Aquí los renombramos a x e y para que sean más interpretables.

**¿Qué hace todo esto?**

- Define árboles que combinan operaciones aritméticas (+, -, *), comparaciones (>), y condicionales (if).

- Reciben dos entradas (x, y) y devuelven un resultado numérico que interpretaremos como acción.



In [ ]:
# Queremos dos objetivos: minimizar error y minimizar complejidad
creator.create("FitnessMulti", base.Fitness, weights=(-1.0, -1.0))  # Minimizar ambos

📌 Aquí definimos un tipo de fitness multiobjetivo. Los pesos negativos indican que queremos minimizar:

- El primer objetivo (por ejemplo, distancia al objetivo).

- El segundo objetivo (por ejemplo, tamaño del programa).



In [ ]:
creator.create("Individual", gp.PrimitiveTree, fitness=creator.FitnessMulti)

📌 Creamos la clase Individual, que será un árbol GP (PrimitiveTree) con una función de fitness multiobjetivo. Es la estructura básica que usará el algoritmo evolutivo.

In [ ]:
toolbox = base.Toolbox()

📌 La toolbox de DEAP es un contenedor donde se registran funciones para:

- Crear individuos y poblaciones.

- Cruzar, mutar, seleccionar, evaluar, etc.

In [ ]:
# Cómo generar individuos
toolbox.register("expr", gp.genHalfAndHalf, pset=pset, min_=1, max_=3)

📌 Se registra un método de generación de expresiones aleatorias (subárboles) usando el método Half and Half:

- Combina crecimiento completo (full) y aleatorio (grow).

- min_ y max_ definen la profundidad del árbol generado.

In [ ]:
toolbox.register("individual", tools.initIterate, creator.Individual, toolbox.expr)

📌 Define cómo crear un individuo: toma una expresión (expr) y la convierte en un creator.Individual.

In [ ]:
toolbox.register("population", tools.initRepeat, list, toolbox.individual)

📌 Crea una población: una lista de individuos generados con la función anterior.

In [ ]:
toolbox.register("compile", gp.compile, pset=pset)

📌 Esta función compila un árbol GP en una función de Python ejecutable. Por ejemplo:



```
compiled_fn = toolbox.compile(individual)
action = compiled_fn(x, y)
```



##### Definición del entorno (entorno simulado tipo grid)

In [ ]:
# Parámetros del entorno
GRID_SIZE = 10
TARGET = (9, 9)
START = (0, 0)

📌 Definimos una cuadrícula 10x10:

- El robot/individuo empieza en (0, 0).

- El objetivo es llegar a (9, 9).

In [ ]:
# Movimientos posibles: 0 = derecha, 1 = arriba
def move(x, y, action):
    if action == 0:   # Avanzar horizontalmente
        x = min(GRID_SIZE - 1, x + 1)
    elif action == 1:  # Subir verticalmente
        y = min(GRID_SIZE - 1, y + 1)
    return x, y

📌 Esta función simula el movimiento de un agente:

- Acción 0: se mueve a la derecha (eje x).

- Acción 1: se mueve hacia arriba (eje y).

- Los movimientos están limitados al tamaño del grid para que no se salga.

##### 🔍 Evaluación de los individuos

In [ ]:
def eval_robot(individual):
    func = toolbox.compile(expr=individual)  # Compila el árbol GP en una función Python
    x, y = START  # Posición inicial
    visited = set()

    # Simular hasta 20 pasos
    for _ in range(20):
        action = func(x, y)  # Ejecuta la función simbólica con x, y
        action = int(action) % 2  # Asegura que la acción sea 0 o 1 (derecha o arriba)
        x, y = move(x, y, action)  # Realiza el movimiento
        visited.add((x, y))

        if (x, y) == TARGET:  # Si llega a la meta, se detiene
            break

    # Objetivo 1: distancia a la meta (a menor, mejor)
    distance_error = abs(x - TARGET[0]) + abs(y - TARGET[1])

    # Objetivo 2: complejidad del árbol (cantidad de nodos, menor es mejor)
    complexity = len(individual)

    return distance_error, complexity

📌 Esta función define la función de fitness, con dos objetivos a minimizar:

- Qué tan lejos quedó el robot del objetivo.

- Qué tan grande es el árbol simbólico (más nodos = más complejo).

##### 🧰 Registro de operaciones en la toolbox

In [ ]:
toolbox.register("evaluate", eval_robot)
toolbox.register("select", tools.selNSGA2)
toolbox.register("mate", gp.cxOnePoint)
toolbox.register("mutate", gp.mutUniform, expr=toolbox.expr, pset=pset)

📌 Se registran los operadores evolutivos:

- evaluate: función de evaluación definida arriba.

- select: selección por NSGA-II (algoritmo MOEA).

- mate: cruce por punto (intercambia subárboles entre padres).

- mutate: mutación por reemplazo aleatorio de un subárbol.

##### ⛔ Límites en tamaño de árboles

In [ ]:
toolbox.decorate("mate", gp.staticLimit(key=len, max_value=17))
toolbox.decorate("mutate", gp.staticLimit(key=len, max_value=17))

📌 decoraciones limitan el tamaño máximo de los árboles generados por cruce o mutación a 17 nodos, evitando explosión de complejidad (bloat).

##### 🌱 Creación de población inicial y estadísticas

In [ ]:
random.seed(42)
pop = toolbox.population(n=100)  # Población inicial de 100 individuos
hof = tools.ParetoFront()        # Conjunto de soluciones no dominadas (frente de Pareto)

# Registro de estadísticas
stats = tools.Statistics(lambda ind: ind.fitness.values)
stats.register("avg", np.mean, axis=0)
stats.register("min", np.min, axis=0)
stats.register("max", np.max, axis=0)

📌 Aquí:

- Se fija una semilla para reproducibilidad.

- Se crea la población inicial.

- Se define el hall of fame (hof): almacena las mejores soluciones (no dominadas).

- Se recogen estadísticas por generación (promedio, mínimo y máximo).



##### ⚙️ Ejecución del algoritmo evolutivo

In [ ]:
pop, log = algorithms.eaMuPlusLambda(
    pop, toolbox,
    mu=100, lambda_=200,       # mu = padres, lambda = hijos generados
    cxpb=0.5, mutpb=0.2,       # prob. cruce y mutación
    ngen=20,                   # número de generaciones
    stats=stats,
    halloffame=hof,
    verbose=True
)

📌 Esto ejecuta el algoritmo evolutivo μ + λ, donde:

- En cada generación:

  - Se seleccionan μ individuos para ser padres.

  - Se generan λ hijos.

  - Se combinan padres e hijos.

  - Se seleccionan los μ mejores para la siguiente generación.

**📊 stats recolecta datos por generación, y hof guarda el frente de Pareto (las mejores soluciones en cuanto a los dos objetivos).**

##### 🎯 Graficar la frontera de Pareto

In [ ]:
fitnesses = [ind.fitness.values for ind in hof]
errors = [f[0] for f in fitnesses]
complexities = [f[1] for f in fitnesses]

📌 Extraemos los valores de fitness del Hall of Fame (hof), que contiene los mejores individuos no dominados en términos de:

- error (distancia a la meta).

- complejidad (número de nodos en el árbol).

📌 Esto genera un diagrama de dispersión donde cada punto representa un programa simbólico:

- Cuanto más abajo y a la izquierda esté, mejor (más preciso y más simple).

- Te permite ver el trade-off entre precisión y complejidad.

In [ ]:
plt.figure(figsize=(8, 6))
plt.scatter(errors, complexities, c="blue")
plt.xlabel("Distancia a la meta (error)")
plt.ylabel("Complejidad del programa (nodos)")
plt.title("Frontera de Pareto: Precisión vs. Simplicidad")
plt.grid(True)
plt.show()

##### 🤖 Simular la trayectoria del mejor individuo

In [ ]:
best = hof[0]
best_func = toolbox.compile(expr=best)

📌 Seleccionamos el primer individuo del Hall of Fame, lo compilamos en una función de Python para ejecutarla.

In [ ]:
def simulate_trajectory(controller_func):
    x, y = START
    path = [(x, y)]

    for _ in range(20):
        action = controller_func(x, y)
        action = int(action) % 2
        x, y = move(x, y, action)
        path.append((x, y))
        if (x, y) == TARGET:
            break
    return path


📌 Esta función simula el movimiento del robot en el grid:

- Ejecuta el programa simbólico 20 veces.

- Guarda la secuencia de posiciones visitadas.

- Detiene la simulación si llega a la meta.



In [ ]:
def plot_trajectory(path):
    grid = np.zeros((GRID_SIZE, GRID_SIZE))
    for (x, y) in path:
        grid[y][x] = 0.5  # Marcar trayectoria

    tx, ty = TARGET
    grid[ty][tx] = 1  # Marcar la meta

    plt.figure(figsize=(6,6))
    plt.imshow(grid, cmap='Greys', origin='lower')
    plt.title("Trayectoria del robot")
    plt.plot([x for x, y in path], [y for x, y in path], marker='o', color='red')
    plt.grid(True)
    plt.show()

📌 Esta función visualiza la trayectoria del robot en la cuadrícula:

- Marca el camino con valores 0.5.

- Resalta la meta con un valor 1.

- Superpone una línea con los pasos realizados.

In [ ]:
# Ejecutamos todo
path = simulate_trajectory(best_func)
plot_trajectory(path)

### Implementación con obstáculos

##### 🧱 1. Generación de obstáculos

In [ ]:
# Generamos obstáculos sin bloquear START o TARGET
obstacles = set()
while len(obstacles) < 15:
    ox, oy = random.randint(0, GRID_SIZE-1), random.randint(0, GRID_SIZE-1)
    if (ox, oy) != START and (ox, oy) != TARGET:
        obstacles.add((ox, oy))

📌 Genera 15 obstáculos aleatorios que no bloquean el punto de partida ni el objetivo. Se almacenan en un set para acceso rápido.

##### 🔁 2. Función de movimiento con obstáculos

In [ ]:
def move_with_obstacles(x, y, action):
    next_x, next_y = x, y
    if action == 0:
        next_x = min(GRID_SIZE - 1, x + 1)
    elif action == 1:
        next_y = min(GRID_SIZE - 1, y + 1)

    # Solo moverse si no hay obstáculo
    if (next_x, next_y) not in obstacles:
        return next_x, next_y
    else:
        return x, y

📌 Similar a move(), pero bloquea el paso si la siguiente celda tiene un obstáculo. El robot se queda en su lugar si el movimiento está obstruido.

##### 📏 3. Nueva función de evaluación

In [ ]:
def eval_robot_with_obstacles(individual):
    func = toolbox.compile(expr=individual)
    x, y = START
    visited = set()

    for _ in range(20):
        action = func(x, y)
        action = int(action) % 2
        x, y = move_with_obstacles(x, y, action)
        visited.add((x, y))
        if (x, y) == TARGET:
            break

    distance_error = abs(x - TARGET[0]) + abs(y - TARGET[1])
    complexity = len(individual)

    return distance_error, complexity

📌 Igual que antes, pero ahora usa move_with_obstacles. Evalúa:

- Qué tan cerca llega el programa del objetivo.

- Cuán simple es la expresión generada.

##### 🔁 4. Re-entrenamiento con obstáculos

In [ ]:
# Re-registramos
toolbox.register("evaluate", eval_robot_with_obstacles)

# Nueva población
pop = toolbox.population(n=100)
hof = tools.ParetoFront()

# Volvemos a correr la evolución con el entorno con obstáculos
pop, log = algorithms.eaMuPlusLambda(
    pop, toolbox,
    mu=100, lambda_=200,
    cxpb=0.5, mutpb=0.2,
    ngen=20, stats=stats,
    halloffame=hof, verbose=True
)


📌 Se redefine la función de evaluación, y se entrena una nueva población desde cero. Los mejores individuos ahora deben sortear obstáculos para llegar al objetivo.

📌 Visualiza la nueva frontera. **La presencia de obstáculos puede generar mayor error o árboles más complejos, ya que el entorno es más difícil.**

In [ ]:
fitnesses = [ind.fitness.values for ind in hof]
errors = [f[0] for f in fitnesses]
complexities = [f[1] for f in fitnesses]

plt.figure(figsize=(8, 6))
plt.scatter(errors, complexities, c="green")
plt.xlabel("Distancia a la meta (error)")
plt.ylabel("Complejidad del programa (nodos)")
plt.title("Frontera de Pareto con obstáculos")
plt.grid(True)
plt.show()

##### 🧭 6. Simular y visualizar trayectoria con obstáculos

In [ ]:
def simulate_with_obstacles(controller_func):
    x, y = START
    path = [(x, y)]

    for _ in range(20):
        action = controller_func(x, y)
        action = int(action) % 2
        x, y = move_with_obstacles(x, y, action)
        path.append((x, y))
        if (x, y) == TARGET:
            break
    return path

📌 El robot ejecuta la lógica del programa en un entorno donde algunos movimientos son bloqueados, y la trayectoria se traza junto con los obstáculos y la meta.

In [ ]:
def plot_path_with_obstacles(path, obstacles):
    grid = np.zeros((GRID_SIZE, GRID_SIZE))
    for (x, y) in path:
        grid[y][x] = 0.5  # trayectoria
    for (ox, oy) in obstacles:
        grid[oy][ox] = 0.8  # obstáculo
    tx, ty = TARGET
    grid[ty][tx] = 1  # meta

    plt.figure(figsize=(6,6))
    plt.imshow(grid, cmap='gray_r', origin='lower')
    plt.plot([x for x, y in path], [y for x, y in path], marker='o', color='red', linewidth=2)
    plt.title("Trayectoria del robot con obstáculos")
    plt.grid(True)
    plt.show()

In [ ]:
best = hof[0]
best_func = toolbox.compile(expr=best)
path = simulate_with_obstacles(best_func)
plot_path_with_obstacles(path, obstacles)

##### 🧠 7. Visualizar el árbol simbólico

In [ ]:
print(best)

📌 Imprime y visualiza con Graphviz la estructura simbólica del mejor programa. Esto permite ver qué lógica de decisión simbólica se ha creado evolutivamente.

In [ ]:
import graphviz
from deap import gp

def draw_tree(individual):
    nodes, edges, labels = gp.graph(individual)
    dot = graphviz.Digraph(format='png')

    # Convertir todos los labels a string explícitamente
    for node_id in nodes:
        label = str(labels[node_id])  # <-- conversión aquí
        dot.node(str(node_id), label)

    for parent, child in edges:
        dot.edge(str(parent), str(child))

    return dot

In [ ]:
draw_tree(best)

## Teoría asociada

##### 🧠 1. Programación Genética (GP)




**¿Qué es la Programación Genética?**

La programación genética es una técnica de inteligencia artificial y optimización evolutiva en la que se buscan programas (expresiones simbólicas) que realicen una tarea. A diferencia de otras formas de aprendizaje automático, GP no entrena pesos ni ajusta parámetros, sino que evoluciona árboles de funciones y operadores que representan programas funcionales.

**¿Cómo funciona?**

1. Se define un conjunto de primitivas (funciones como +, *, if_then_else) y terminales (valores constantes y variables como x, y).

2. Se crean árboles sintácticos que combinan primitivas y terminales.

3. Cada árbol es una posible solución (por ejemplo, una función de control que decide si moverse arriba o a la derecha).

4. A través de mecanismos evolutivos como cruce, mutación y selección, se generan nuevas poblaciones.

5. Se seleccionan las mejores soluciones en cada generación según una función de evaluación (fitness).

**¿Por qué árboles?**

Porque representan de forma natural expresiones matemáticas o lógicas. Un árbol puede representar algo como:



```
if x > y:
    return x + 1
else:
    return y - 1
```




##### **⚖️ 2. Optimización Multiobjetivo**

**¿Qué es la optimización multiobjetivo?**

Muchos problemas reales no tienen un solo objetivo. En este caso, queremos que el programa:

- 🏁 Lleve al robot lo más cerca posible del objetivo (minimizar error).

- 🧠 Sea lo más simple posible (minimizar la complejidad del árbol).

En lugar de combinar ambos criterios en un único valor de fitness (como una suma ponderada), usamos un enfoque multiobjetivo para mantener el equilibrio entre ambos.


**¿Qué es la frontera de Pareto?**

**Es el conjunto de soluciones óptimas no dominadas.** Una solución A domina a una solución B si es mejor en todos los objetivos o igual en algunos y mejor en al menos uno. **La frontera de Pareto es el conjunto de soluciones que:**

- No son superadas simultáneamente por ninguna otra.

- Representan distintos compromisos entre los objetivos.

En tu caso:

- Un punto cerca del eje X tiene bajo error pero es complejo.

- Un punto cerca del eje Y es muy simple pero menos preciso.

**Esto permite al usuario elegir según la necesidad (ej., en robótica puede preferirse simplicidad para eficiencia de ejecución).**

##### 🧬 3. Algoritmo Evolutivo Mu + Lambda (eaMuPlusLambda)

Este es el algoritmo evolutivo que usamos, caracterizado por:

**Parámetros:**
- mu: número de individuos seleccionados como padres (por ejemplo, 100).

- lambda_: número de hijos generados (por ejemplo, 200).

- cxpb: probabilidad de cruce.

- mutpb: probabilidad de mutación.


**Flujo:**

1. Se seleccionan mu padres.

2. Se generan lambda hijos mediante cruce y mutación.

3. Se combinan padres e hijos (mu + lambda individuos).

4. Se seleccionan los mejores mu individuos para la siguiente generación (según su fitness multiobjetivo).

**Este método permite un equilibrio entre exploración (hijos nuevos) y explotación (conservar lo mejor de lo anterior).**


##### 🧩 4. Compilación y ejecución simbólica

Gracias a la función compile de DEAP, cada árbol generado puede convertirse en una función de Python ejecutable. Esto es clave porque:

- Permite evaluar directamente la expresión en el entorno (por ejemplo, decidir una acción basada en x, y).

- Es completamente flexible: puedes insertar cualquier lógica en la expresión.

Esto convierte al GP en una herramienta generadora de lógica interpretable.

##### 🧱 5. Simulación del entorno con obstáculos

**Entorno básico:**

- Una cuadrícula de 10x10.

- El agente comienza en (0, 0) y busca llegar a (9, 9).

**Obstáculos:**

- Se agregan celdas que bloquean el movimiento del agente.

- Esto fuerza al programa a aprender a esquivar obstáculos (aunque sin verlos directamente).

**Movimiento:**

- Acción 0: moverse a la derecha (x + 1).

- Acción 1: moverse hacia arriba (y + 1).

- Si hay un obstáculo, el agente se queda quieto.

**Esto añade dificultad estructural, y pone a prueba la capacidad de los árboles para generalizar un comportamiento útil.**

##### 🧠 6. Evaluación de soluciones

Cada individuo es evaluado con:

**Objetivo 1: Distancia al objetivo**

```
abs(x - TARGET[0]) + abs(y - TARGET[1])
```

Esto mide qué tan cerca llega el agente a su meta tras 20 pasos.

**Objetivo 2: Complejidad del árbol**

```
len(individual)
```

Esto mide cuántos nodos tiene el árbol. Menos nodos = menor complejidad computacional = mayor interpretabilidad.



##### 📊 7. Visualización de resultados

**Frontera de Pareto:**

- Gráfica de error vs. complejidad.

- Te permite analizar los compromisos entre objetivos.

**Trayectoria del robot:**

- Muestra cómo el mejor árbol se comporta en el entorno.

- Útil para evaluar acciones simbólicas aprendidas.

**Árbol GP visual:**

- Muestra gráficamente cómo es la estructura del programa.

- Útil para inspeccionar lógica y verificar si tiene sentido.

##### 🔎 8. Interpretabilidad y explicabilidad

A diferencia de redes neuronales u otros métodos de caja negra, la GP produce expresiones que puedes entender, imprimir y visualizar. Esto es:

- Transparente.

- Analizable.

- Auditado fácilmente.

**En entornos sensibles como salud, robótica o educación, esto puede ser una ventaja clave.**